# <font color=blue>Stacking </font>##

**Importing necessary Libraries**

In [2]:
import numpy as np
import pandas as pd

**Importing dataset**

In [4]:
data = pd.read_csv('C:/Abhishek/ML-Algorithms/AV/stacking/data_cleaned.csv')
data.head()

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,7.2500,0,0,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
1,1,38.0,71.2833,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
2,1,26.0,7.9250,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
3,1,35.0,53.1000,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4,0,35.0,8.0500,0,0,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1


In [6]:
#size of the data
data.shape

(891, 25)

In [7]:
#checking mthe missing values

data.isnull().sum()

Survived      0
Age           0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
SibSp_0       0
SibSp_1       0
SibSp_2       0
SibSp_3       0
SibSp_4       0
SibSp_5       0
SibSp_8       0
Parch_0       0
Parch_1       0
Parch_2       0
Parch_3       0
Parch_4       0
Parch_5       0
Parch_6       0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

**Separating dependent & the independent variables**

In [9]:
#independent variable

x = data.drop(['Survived'],axis = 1)

#dependent variable

y = data['Survived']

x.shape, y.shape

((891, 24), (891,))

**Making a Test & Training set**

In [13]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y = train_test_split(x,y,random_state = 9,stratify = y)

print(train_x.shape),
print(test_x.shape),
print(train_y.shape),
print(test_y.shape)

(668, 24)
(223, 24)
(668,)
(223,)


**Base Models**

In [17]:
#importing predictive models

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

**Model Training & Predictions**

In [18]:
def model_predictions(model,train_x,train_y,test_x):
    
    #train the model
    model.fit(train_x,train_y)
    
    #storing the predictions for train & test
    
    pred_train = model.predict(train_x)
    pred_test = model.predict(test_x)
    return pred_train,pred_test

In [21]:
#Model 1 - Decision Trees

DT = DecisionTreeClassifier(random_state = 101)
M1_train,M1_test = model_predictions(DT,train_x,train_y,test_x)

In [23]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X = train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [24]:
#Model 2 - Logistic Regression

LR = LogisticRegression(random_state = 101)
M2_train,M2_test = model_predictions(LR,train_x,train_y,test_x)

In [26]:
#Model 3 - K Nearest Neighbour

knn = KNeighborsClassifier()
M3_train,M3_test = model_predictions(knn,train_x,train_y,test_x)

**Stacking The Model**

In [28]:
#Creating a New train dataframe

train_predictions = {
    'DT': M1_train,
    'LR': M2_train,
    'knn': M3_train
}
train_predictions = pd.DataFrame(train_predictions)
train_predictions.head()

,DT,LR,knn
0,1,1,0
1,1,1,1
2,1,1,0
3,0,1,1
4,0,0,0


In [29]:
#Craeting a New test dataframe

test_predictions = {
    'DT': M1_test,
    'LR': M2_test,
    'knn': M3_test
}
test_predictions = pd.DataFrame(test_predictions)
test_predictions.head()

,DT,LR,knn
0,0,0,0
1,1,1,1
2,0,0,0
3,1,0,0
4,0,0,0


In [30]:
#Stacked Model

model = LogisticRegression()

model.fit(train_predictions,train_y)
model.score(test_predictions,test_y)

0.726457399103139